In [1]:
import pandas as pd
import torch
from torch import nn
from torchsummary import summary
from collections import OrderedDict
from torchvision.models import resnet50, densenet121, inception_v3
from tqdm import tqdm
from skimage.io import imread
import numpy as np
from skimage.transform import resize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
from torchvision import models, transforms
import os
import cv2
import fnmatch


### First, run runmodels.sh for all model combos. Then run the following cells to benchmark the acl and breast Inception models with/ without RadImageNet pretraining. 

In [2]:

def eval_model(data_dir, database, model_name, structure, verbose=False):
    """ 
    Grab lowest validation loss epoch of training history across all folds
    for the specified model. 

    Average loss and AUC metrics for this validation-selected model and report.
    """
    file_pattern = f"training_history_{model_name}_{database}_fold_*_structure_{structure}.csv"
    data_dir = os.path.join("..", "data", data_dir, "models")
    fold_dfs = []

    if verbose:
        print(f"Looking for files in: {data_dir}")
        print(f"File pattern: {file_pattern}")

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                file_path = os.path.join(root, file)
                if verbose:
                    print(f"Processing file: {file_path}")
                df = pd.read_csv(file_path)
                # Find the row with the max validation AUC
                max_val_auc_row = df.loc[df['val_auc'].idxmax()]
                fold_dfs.append(max_val_auc_row)
    
    if len(fold_dfs) == 0:
        if verbose:
            print(f"No files matched the pattern {file_pattern} in {data_dir}")
        return pd.Series(dtype=float)
    
    combined_df = pd.DataFrame(fold_dfs)
    avg_metrics = combined_df.mean()


    print("\nAverage Metrics Across Folds:")
    print(avg_metrics)
    print()

    return avg_metrics

results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

params_list = [
    {"data_dir": "acl", "database": "RadImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "acl", "database": "ImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "breast", "database": "RadImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "breast", "database": "ImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "acl", "database": "RadImageNet", "model_name": "InceptionV3", "structure": "unfreezetop2"},
    {"data_dir": "acl", "database": "RadImageNet", "model_name": "ResNet50", "structure": "freezeall"},
]

for params in params_list:
    print(f"Processing: {params}")
    avg_metrics = eval_model(**params, verbose=True)
    
    result_filename = f"results_{params['data_dir']}_{params['database']}_{params['model_name']}_{params['structure']}.txt"
    result_filepath = os.path.join(results_dir, "pytorch", result_filename)
    
    with open(result_filepath, 'w') as result_file:
        result_file.write("Averaged Metrics Across Folds:\n")
        result_file.write(avg_metrics.to_string())
    
    print(f"Averaged metrics for {params} saved to {result_filepath}")
    print("="*50)
    print()


Processing: {'data_dir': 'acl', 'database': 'RadImageNet', 'model_name': 'InceptionV3', 'structure': 'freezeall'}
Looking for files in: ../data/acl/models
File pattern: training_history_InceptionV3_RadImageNet_fold_*_structure_freezeall.csv
Processing file: ../data/acl/models/training_history_InceptionV3_RadImageNet_fold_2_structure_freezeall.csv
Processing file: ../data/acl/models/training_history_InceptionV3_RadImageNet_fold_5_structure_freezeall.csv
Processing file: ../data/acl/models/training_history_InceptionV3_RadImageNet_fold_1_structure_freezeall.csv
Processing file: ../data/acl/models/training_history_InceptionV3_RadImageNet_fold_3_structure_freezeall.csv
Processing file: ../data/acl/models/training_history_InceptionV3_RadImageNet_fold_4_structure_freezeall.csv

Average Metrics Across Folds:
train_loss    0.666481
val_loss      0.648277
train_auc     0.619436
val_auc       0.761468
dtype: float64

Averaged metrics for {'data_dir': 'acl', 'database': 'RadImageNet', 'model_name'

In [3]:
import os
import pandas as pd
import numpy as np
import fnmatch

def eval_tflow_model(data_dir, database, model_name, structure, verbose=False):
    """ 
    Modified for diff filestructure with tflow models - sry for code bloat
    
    Grab lowest validation loss epoch of training history across all folds
    for the specified model. 

    Average loss and AUC metrics for this validation-selected model and report.
    """
    file_pattern = f"{data_dir}-{structure}-fold*-{database}-{model_name}-256-256-0.001.csv"
    data_dir = os.path.join("..", "tflow_replicated_expts", data_dir, "loss")

    fold_dfs = []

    if verbose:
        print(f"Looking for files in: {data_dir}")
        print(f"File pattern: {file_pattern}")

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                file_path = os.path.join(root, file)
                if verbose:
                    print(f"Processing file: {file_path}")
                df = pd.read_csv(file_path)
                # Find the row with the max validation AUC
                max_val_auc_row = df.loc[df['val_auc'].idxmax()]
                fold_dfs.append(max_val_auc_row)
    
    if len(fold_dfs) == 0:
        if verbose:
            print(f"No files matched the pattern {file_pattern} in {data_dir}")
        return pd.Series(dtype=float)
    
    combined_df = pd.DataFrame(fold_dfs)
    avg_metrics = combined_df.mean()
    
    # Reorder the metrics
    ordered_metrics = avg_metrics[['train_loss', 'val_loss', 'train_auc', 'val_auc']]

    if verbose:
        print("Average Metrics Across Folds:")
        print(ordered_metrics)

    return ordered_metrics

results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

params_list = [
    {"data_dir": "acl", "database": "RadImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "acl", "database": "ImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "breast", "database": "RadImageNet", "model_name": "InceptionV3", "structure": "freezeall"},
    {"data_dir": "breast", "database": "ImageNet", "model_name": "InceptionV3", "structure": "freezeall"}
]

for params in params_list:
    print(f"Processing: {params}")
    avg_metrics = eval_tflow_model(**params, verbose=True)
    
    result_filename = f"results_{params['data_dir']}_{params['database']}_{params['model_name']}_{params['structure']}.txt"
    result_filepath = os.path.join(results_dir, "tflow", result_filename)
    
    with open(result_filepath, 'w') as result_file:
        result_file.write("Averaged Metrics Across Folds:\n")
        result_file.write(avg_metrics.to_string())
    
    print(f"Averaged metrics for {params} saved to {result_filepath}")
    print("="*50)


Processing: {'data_dir': 'acl', 'database': 'RadImageNet', 'model_name': 'InceptionV3', 'structure': 'freezeall'}
Looking for files in: ../tflow_replicated_expts/acl/loss
File pattern: acl-freezeall-fold*-RadImageNet-InceptionV3-256-256-0.001.csv
Processing file: ../tflow_replicated_expts/acl/loss/acl-freezeall-fold1-RadImageNet-InceptionV3-256-256-0.001.csv
Processing file: ../tflow_replicated_expts/acl/loss/acl-freezeall-fold3-RadImageNet-InceptionV3-256-256-0.001.csv
Processing file: ../tflow_replicated_expts/acl/loss/acl-freezeall-fold2-RadImageNet-InceptionV3-256-256-0.001.csv
Processing file: ../tflow_replicated_expts/acl/loss/acl-freezeall-fold4-RadImageNet-InceptionV3-256-256-0.001.csv
Processing file: ../tflow_replicated_expts/acl/loss/acl-freezeall-fold5-RadImageNet-InceptionV3-256-256-0.001.csv
Average Metrics Across Folds:
train_loss    0.615819
val_loss      0.607071
train_auc     0.798664
val_auc       0.833382
dtype: float64
Averaged metrics for {'data_dir': 'acl', 'data